# import các thư viện

In [29]:
import cv2
import torch
import numpy as np
from skimage import measure
import os
import pandas as pd


# dùng model là yolov5

In [30]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/best.pt')

Using cache found in C:\Users\dangt/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-20 Python-3.10.0 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


# hàm xuất ra ma trận nhị phân đặc trưng cho kí tự

In [31]:
# Xuất ra ma trận nhị phân ( đen = 0, trắng = 255)
def feature(folder_path, file):
    binaries = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.png'):
            # Đọc tệp tin ảnh
            img = cv2.imread(os.path.join(folder_path, filename), cv2.IMREAD_GRAYSCALE)

            # Đổi ảnh sang ma trận nhị phân
            ret, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

            # Thêm ma trận nhị phân vào mảng
            binaries.append(binary)
        

    # Khởi tạo mảng pixel trắng đen có kích thước giống với các ma trận trong binaries với các phần tử được khởi tạo là 0
    black_pixels = np.zeros_like(binaries[0])
    white_pixels = np.zeros_like(binaries[0])
    
    # Tính toán số lượng pixel màu đen và màu trắng cho từng pixel trong các ma trận
    for binary in binaries:
        black_pixels += binary == 0
        white_pixels += binary == 255

    # So sánh số lượng pixel màu đen và màu trắng để quyết định màu sắc cuối cùng của pixel trong ma trận đặc trưng
    feature = np.zeros_like(binaries[0], dtype=np.uint8)
    for i in range(feature.shape[0]):
        for j in range(feature.shape[1]):
            if black_pixels[i][j] > white_pixels[i][j]:
                feature[i][j] = 0
            else:
                feature[i][j] = 255

    return feature

# xuất ra ma trận đặc trưng cho tất cả các kí tự

In [32]:
features = []
folder_path = "char/"
files = os.listdir(folder_path)
for file in files:
    if os.path.isdir(os.path.join(folder_path, file)):
        folder_path_ii = os.path.join(folder_path, file)
        features.append(feature(folder_path_ii, file))

In [33]:
features2 = []
folder_path = "char2line/"
files = os.listdir(folder_path)
for file in files:
    if os.path.isdir(os.path.join(folder_path, file)):
        folder_path_ii = os.path.join(folder_path, file)
        features2.append(feature(folder_path_ii, file))

# so sánh để đưa ra kết luận

In [34]:
def predict(binary, number, features):
    n = 0
    max = 0
    i = 0

    # number == 1: kí tự đang đọc là kí tự chữ số
    if (number == 1):

        # Đọc từ 0 đến 9
        for i in range (10):

            # Tính số pixel giống nhau của chữ số cần dự đoán và đặc trưng của các chữ số
            matching_pixels = np.sum(binary == features[i])
            total_pixels = binary.shape[0] * binary.shape[1]

            # Tính % giống nhau bằng cách lấy số pixel giống nhau / tổng số pixel * 100
            matching_percentage = matching_pixels / total_pixels * 100

            # Lấy max = tỉ lệ khớp nhất của chữ số cần dự đoán với các chữ số đặc trưng, n là chữ số được dự đoán
            if (matching_percentage>max): 
                n = i
                max = matching_percentage

    # number == 0: kí tự đang đọc là kí tự chữ cái
    else:

        # Nếu không phải chữ số thì đọc các feature chữ cái
        for i in range(10, len(features)):
            matching_pixels = np.sum(binary == features[i])
            total_pixels = binary.shape[0] * binary.shape[1]
            matching_percentage = matching_pixels / total_pixels * 100
            if (matching_percentage>max): 
                n = i
                max = matching_percentage
    # print(n,max)
    if (n == 7 and max < 65): 
        n = 1
    # print(n, max)
    return n, max

In [82]:
def read_char(labels_list, max, type):
    sorted_list = sorted(labels_list, key=lambda x: x[0])
    if (len(labels_list) > max):
        delete = len(labels_list) - max
    else:
        delete = 0
    predicted = []
    if (type == 0):
        for i in range(len(sorted_list)):
            x, dg, y = sorted_list[i]
            predicted.append((i, predict(dg, 1, features), predict(dg, 0, features)))
    
    else:
        for i in range(len(sorted_list)):
            x, dg, y = sorted_list[i]
            predicted.append((i, predict(dg, 1, features2), predict(dg, 0, features2)))
    predicted = sorted(predicted, key=lambda x: x[1][1], reverse=True)
    predicted_string = ""
    j = 0
    lst = [list(item) for item in predicted]
    # print(lst)
    if (type == 0):
        for i in reversed(range(len(predicted))):
            if (j == delete):
                break
            if (lst[i][0] < 2):
                lst.remove(lst[i])
                j = j + 1
            elif (lst[i][0] > (2+ delete)):
                lst.remove(lst[i])
                j = j + 1
            else:
                if (lst[i][2][1] >= 65):
                    lst.remove(lst[i-1])
                    j = j + 1
                else:
                    lst.remove(lst[i])
                    j = j + 1
    if (type == 1):
        for i in reversed(range(len(predicted))):
            if (j == delete):
                break
            # print(lst[i])
            if (lst[i][0] < 2):
                lst.remove(lst[i])
                j = j + 1
            else:
                if (lst[i][2][1] >= 65):
                    # print(lst[i-1])
                    lst.remove(lst[i-1])
                    j = j + 1
                else:
                    # print(lst[i])
                    lst.remove(lst[i])
                    j = j + 1
    if (type == 2):
        if (delete != 0):
            for i in range (delete):
                lst.pop(-1)

    predicted = [tuple(item) for item in lst] # chuyển lại thành tuple
    i = 0
    # print(predicted)
    for item in sorted(lst, key=lambda x: x[0]):
            i = i + 1
            # print(item)
            if (i == 3 and type == 0):
                if (item[2][0] == 10):
                    predicted_string += "A"
                if (item[2][0] == 11):
                    predicted_string += "B"
                if (item[2][0] == 12):
                    predicted_string += "C"
                if (item[2][0] == 13):
                    predicted_string += "D"
                if (item[2][0] == 14):
                    predicted_string += "F"
                if (item[2][0] == 15):
                    predicted_string += "G"
                if (item[2][0] == 16):
                    predicted_string += "L"
                if (item[2][0] == 17):
                    predicted_string += "N"
                if (item[2][0] == 18):
                    predicted_string += "S"
                if (item[2][0] == 19):
                    predicted_string += "V"
                if (item[2][0] == 20):
                    predicted_string += "Y"
                if (item[2][0] == 21):
                    predicted_string += "Z"
            elif (i==3 and type == 1):
                if (item[2][0] == 10):
                    predicted_string += "A"
                if (item[2][0] == 11):
                    predicted_string += "E"
            else:
                predicted_string += str(item[1][0])
    # print(predicted_string)
    return predicted_string

In [83]:
def read(cropped_image, type):
    gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
    # Sử dụng hàm connectedComponents để tìm các vùng kết nối trên ảnh nhị phân
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    num_labels, labels = cv2.connectedComponents(binary)

    d = 0
        # Sử dụng hàm connectedComponentsWithStats để tính toán diện tích của các vùng kết nối
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
    for i in range(num_labels):
        area = stats[i, cv2.CC_STAT_AREA]
        if area < 90 or area > 1500:
            labels[labels == i] = 0
        else: 
            d = d+1
    
    x_centroids = centroids[:, 0]
    y_centroids = centroids[:, 1]
    
    labels_list = []

    if (d > 5):
        # print(d)
        for i in range(1, num_labels):
            # Lấy thông tin của nhãn hiện tại
            x, y, w, h, area = stats[i]

            # Nếu diện tích của nhãn nằm trong khoảng từ 100 đến 1500
            if 90 < area < 1500:
                # Cắt ảnh của nhãn hiện tại từ ảnh gốc
                digit_img = binary[y:y+h, x:x+w]
                digit_img = cv2.resize(digit_img, (25,60))
                # print(x_centroids[i], y_centroids[i])
                labels_list.append((x_centroids[i], digit_img, y_centroids[i]))   
                # print(labels_list)      
    else:
        return "0"
    
    read_final = None
    if (type == 1):
        max = 8
        read_final = read_char(labels_list, max, 0)
        # print(s)
    else: 
        label_distances = []
        first_label = min(labels_list, key=lambda label: label[0])
        first_x_centroid, first_digit_img, first_y_centroid = first_label
        predicted = predict(first_digit_img, 1,  features2)
        while (len(labels_list) > 8):
            if (predicted[1] < 65):
                for i, label in enumerate(labels_list):
                    if label == first_label:
                        del labels_list[i]
                        break
                first_label = min(labels_list, key=lambda label: label[0])
                first_x_centroid, first_digit_img, first_y_centroid = first_label
                predicted = predict(first_digit_img, 1, features2)
            else:
                break
            
        line1 = []
        line2 = []
        i = 0
        labels_list.sort(key=lambda x: x[0])
        # print(labels_list)
        for label in labels_list:
            x_centroid, digit_img, y_centroid = label
            distance = abs(y_centroid - first_y_centroid)
            distance_x = abs(x_centroid - first_x_centroid)
            label_distances.append(distance)
            if (distance >= 6 + i*1.5):
                line1.append(label)  
            else:
                line2.append(label)
            i = i + 1

        # print(len(line1), len(line2))
        # print(label_distances)
        max_line1 = 3
        max_line2 = 5
        line1 = read_char(line1, max_line1, 1)
        line2 = read_char(line2, max_line2, 2)
        read_final = line1 + line2
        # print(read_final)

    return read_final

# đọc ký tự trên biển số xe

In [37]:
def readPlate(image, model):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image)

    for i, det in enumerate(results.xyxy[0]):
        type = 0
        # Lấy tọa độ bbox của vật thể thứ i
        bbox = det[0:4].cpu().numpy()

        # Cắt lấy vùng ảnh nằm trong bbox
        cropped_image = image[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        # Tính tỉ lệ chiều dài/ chiều rộng của biển số
        ratio = (bbox[2]-bbox[0])/(bbox[3]-bbox[1])
        if (ratio >= 2.2):
            type = 1
            cropped_image = cv2.resize(cropped_image, (256, 100))
        else:
            type = 2
            cropped_image = cv2.resize(cropped_image, (550, 100))
        
        # print("Type: ", type)
        read_fn = read(cropped_image, type)
        # print(read_fn)
        return type, read_fn

# Test xuất ra loại biển số (1 dòng/ 2 dòng)

In [38]:
# folder_path = "yellow_license_plate/" # đường dẫn đến folder chứa ảnh

# type = []
# for filename in os.listdir(folder_path):
#     if filename.endswith(".jpg") or filename.endswith(".png"): 
#         # kiểm tra xem file có phải là ảnh jpg hoặc png không
#         file_path = os.path.join(folder_path, filename)
#         img = cv2.imread(file_path) # đọc ảnh bằng OpenCV
#         # sử dụng ảnh tại đây
#         name, extension = os.path.splitext(filename) # Tách phần đuôi của tên tệp
#         print("Image: ", name)
#         pre_final.append((name, readPlate(img, model)))

# # Tạo DataFrame từ mảng
# df = pd.DataFrame(type, columns=['Image', 'Predicted'])

# # Xuất ra file CSV
# df.to_csv('predicted_output.csv', index=False)


# Test những biển số sai sót

In [84]:
img = cv2.imread("test9.jpg") # đọc ảnh bằng OpenCV
readPlate(img, model)

[(115.36837748344371, array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,  76,   0,   0],
       [  0,   0,   0, ..., 195,   0,   0],
       ...,
       [244, 255, 255, ..., 255, 255, 244],
       [ 96, 255, 255, ..., 255, 255,  96],
       [  0, 255, 255, ..., 255, 255,   0]], dtype=uint8), 70.09850993377484), (193.61354019746122, array([[  0,   0,   0, ..., 255, 122,   0],
       [  0,   0,   0, ..., 255, 172,  15],
       [  0,   0,   0, ..., 255, 249,  39],
       ...,
       [  0,   0,   0, ..., 255, 249, 244],
       [  0,   0,   0, ..., 255, 172,  96],
       [  0,   0,   0, ..., 255, 122,   0]], dtype=uint8), 66.9703808180536), (199.82546494992846, array([[  0,   0,   0, ..., 255, 255,  35],
       [  0,   0,   0, ..., 255, 255,  36],
       [  0,   0,   0, ..., 255, 255,  36],
       ...,
       [  0,   0,   0, ..., 255, 242,  33],
       [  0,   0,   0, ..., 255, 159,  13],
       [  0,   0,   0, ..., 255, 107,   0]], dtype=uint8), 28.722460658082976), (26

(2, '18A21443')

# xuất ra file csv kết quả kiểm thử hình ảnh trong tập kiểm thử

In [52]:
folder_path = "car_long/" # đường dẫn đến folder chứa ảnh

pre_final = []
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        # kiểm tra xem file có phải là ảnh jpg hoặc png không
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path) # đọc ảnh bằng OpenCV
        # sử dụng ảnh tại đây
        name, extension = os.path.splitext(filename) # Tách phần đuôi của tên tệp
        # print(readPlate(img, model))
        type, pre_fn = readPlate(img, model)
        pre_final.append((name, type, pre_fn))

# Tạo DataFrame từ mảng
df = pd.DataFrame(pre_final, columns=['Image', 'Type', 'Predicted'])

# Xuất ra file CSV
df.to_csv('predicted_output.csv', index=False)


198.59459459459458 33.36756756756757
262.44853737811485 34.96641386782232
328.5695695695696 36.44044044044044
121.53418413855971 66.1485870556062
194.1836027713626 69.37759815242494
261.1831275720165 69.57407407407408
364.47768281101617 72.82716049382717
435.5796915167095 77.01156812339332
[32.78101948803863, 31.18217318778388, 29.70814661516576, 0.0, 3.2290110968187378, 3.4254870184678765, 6.678573438220965, 10.862981067787118]
[(0, (3, 82.73333333333333), (11, 49.46666666666666)), (1, (0, 74.93333333333332), (11, 59.46666666666667)), (2, (5, 63.06666666666667), (11, 73.8))]
[(4, (4, 78.73333333333333), (10, 66.93333333333334)), (2, (5, 76.93333333333334), (11, 63.46666666666667)), (3, (9, 75.53333333333333), (11, 53.53333333333333)), (1, (2, 73.73333333333333), (11, 46.06666666666667)), (0, (8, 70.46666666666667), (11, 55.53333333333333))]
232.33203125 47.3671875
207.07736943907156 48.986460348162474
173.15513626834382 52.696016771488466
146.72826086956522 54.84130434782609
118.76694

# đọc video kiểm thử 

In [41]:
# # Tạo một đối tượng VideoCapture để đọc video
# cap = cv2.VideoCapture('test.mp4')

# # Lấy kích thước khung hình của video
# width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# # Tạo đối tượng VideoWriter để ghi video
# fourcc = cv2.VideoWriter_fourcc(*'mp4v')
# out = cv2.VideoWriter('processed_video.mp4', fourcc, 30, (width, height))
# # Đọc từng khung hình của video, xử lý và ghi lại
# result_s = ''
# r = 0

# with open("license_plates_history.txt", "w") as file:
#     file.write('')

# while True:
#     print(1)
#     ret, frame = cap.read()
#     if not ret:
#         break
#     # Xử lý khung hình ở đây
#     processed_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     results = model(processed_frame)
    
#     if len(results.pred[0]) >= 1:
#         if (results.pred[0][0][1].item() <= 50):
#             r = 0
#         for i in range (len(results.pred[0])):
#             accuracy = float(results.pred[0][i][4].item())
#             if (accuracy >= 0.95 and results.pred[0][i][0].item() <= 1500 and r == 0):
#                 if (len(readPlate(processed_frame,model)) >= 7 and len(readPlate(processed_frame,model)) <= 8):
#                     result_s = readPlate(processed_frame,model)
#                     r = 1
#                     with open('license_plates_history.txt', 'a') as file:
#                         file.write(result_s + '\n')
#     # Vẽ chuỗi result_s lên khung hình results
#     cv2.putText(results.render()[0], result_s, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
#     # Ghi khung hình đã xử lý vào đối tượng VideoWriter
#     out.write(cv2.cvtColor(results.render()[0], cv2.COLOR_RGB2BGR))

#     # # Hiển thị khung hình đã xử lý
#     cv2.imshow('Processed Frame', cv2.cvtColor(results.render()[0], cv2.COLOR_RGB2BGR))
#     if cv2.waitKey(1) == ord('q'):
#         break

# # Giải phóng các tài nguyên và đóng các cửa sổ hiển thị
# out.release()
# cv2.destroyAllWindows()

In [55]:

# Đọc file CSV vào DataFrame
df = pd.read_csv('predicted_output.csv')

# Sắp xếp DataFrame theo cột số
df_sorted = df.sort_values(by='Image', ascending=True)

# Lưu lại kết quả vào file CSV
df_sorted.to_csv('predicted_output.csv', index=False)

In [43]:
# import csv

# # Đường dẫn đến file csv
# file_path = 'true_output.csv'

# # Danh sách các dòng mới cần thêm
# new_rows = [
#     ['3', '2', '29E01526'],
#     ['5', '2', '27A04290'],
#     ['6', '2', '27A04290'],
#     ['9', '2', '29E01526'],
#     ['10', '2', '30E82594'],
#     ['11', '2', '29E02000'],
#     ['12', '2', '81A10048'],
#     ['14', '2', '24E00001'],
#     ['20', '2', '81A10048'],
#     ['21', '2', '49E00001'],
#     ['25', '2', '37A74953'],
#     ['26', '2', '18E00003'],
#     ['27', '2', '24E00001'],
#     ['28', '2', '61A88888'],
#     ['37', '2', '61A88888']
# ]

# # Đọc nội dung file csv vào list
# with open(file_path, 'r') as file:
#     reader = csv.reader(file)
#     rows = list(reader)

# # Thêm các dòng mới vào list
# for new_row in new_rows:
#     rows.append(new_row)

# # Sắp xếp lại các cột
# rows = sorted(rows, key=lambda x: (x[0], x[1], x[2]))

# # Ghi lại nội dung list vào file csv
# with open(file_path, 'w', newline='') as file:
#     writer = csv.writer(file)
#     writer.writerows(rows)


In [56]:
# Đọc hai file CSV và lưu chúng vào hai DataFrame
df1 = pd.read_csv('predicted_output.csv')
df2 = pd.read_csv('true_output.csv')

# Kết hợp hai DataFrame và lấy các cột cần thiết
merged_df = pd.merge(df1[['Image', 'Predicted']], df2[['Image', 'True']], on='Image')

merged_df['Diff'] = merged_df.apply(lambda row: 0 if row['Predicted'] == row['True'] else 1, axis=1)

# Thêm cột Predict dựa trên giá trị của cột thứ 2
merged_df['Predict'] = merged_df['Predicted'].apply(lambda x: 0 if x == "0" else 1)

# Thêm cột Miss để kiểm tra độ dài khác nhau giữa hai cột 'Predicted' và 'True'
merged_df['Miss'] = merged_df.apply(lambda row: 1 if (not pd.isnull(row['Predicted'])) and (not pd.isnull(row['True'])) and len(str(row['Predicted'])) - len(str(row['True'])) != 0 else 0, axis=1)

# Lưu các cột vào file CSV
merged_df.to_csv('Evaluate.csv', index=False)




In [57]:
# Đọc file CSV vào DataFrame
merged_df = pd.read_csv('Evaluate.csv')

zero_Predict_ratio = (merged_df['Predict'] == 0).sum() / len(merged_df) * 100
miss_Predict_ratio = (merged_df['Miss'] == 1).sum()/ len(merged_df) * 100
right_Predict_ratio = 100 - ((merged_df['Diff'] == 1).sum()) / len(merged_df) * 100
wrong_Predict = (merged_df['Diff'] == 1).sum()- (merged_df['Predict'] == 0 - (merged_df['Miss'] == 1).sum()).sum()
wrong_Predict_ratio = wrong_Predict / len(merged_df) * 100

print("Số ảnh kiểm thử: ", len(merged_df))
print("Tỉ lệ không thể nhận diện biển số: {:.2f}%".format(zero_Predict_ratio))
print("Tỉ lệ đọc sai biển số: {:.2f}%".format(wrong_Predict_ratio- miss_Predict_ratio))
print("Tỉ lệ đọc thiếu chữ số: {:.2f}%".format(miss_Predict_ratio))
print("Tỉ lệ biển số đọc đúng: {:.2f}%".format(right_Predict_ratio))

mask = (merged_df['Diff'] == 1)
df_filtered = merged_df.loc[mask]
print(df_filtered)

Số ảnh kiểm thử:  489
Tỉ lệ không thể nhận diện biển số: 1.23%
Tỉ lệ đọc sai biển số: 4.29%
Tỉ lệ đọc thiếu chữ số: 3.48%
Tỉ lệ biển số đọc đúng: 92.23%
     Image Predicted      True  Diff  Predict  Miss
6       12   14E1004  81A10048     1        1     1
8       20   14E1004  81A10048     1        1     1
45     509  51A89711  51A89714     1        1     0
49     554    54Y210  54A72110     1        1     1
50     555         0  54A72110     1        0     1
57     647  51F24483  51F24403     1        1     0
67     782  30V39931   30V3993     1        1     1
68     783  30V39931   30V3993     1        1     1
82    1961         0  51A85325     1        0     1
134   3820  51B21666  51D21666     1        1     0
158   5347  51G25182  51G25181     1        1     0
198   6897    51Z210  51A72110     1        1     1
200   6905  51A19642  51A96141     1        1     0
212   7078  28A51796  29A51796     1        1     0
221   7214   15A5380  51F15380     1        1     1
222   7215   15